# Excelで入力した教師データの拡張

編集日：2023/12/03 <br>
編集者：[nogikun](https://github.com/nogikun) <br>

#### 期待するファイル
以下のようなテーブルを含むExcelファイルを用意してください。<br>
シート数は１つで、シート名は任意です。

※ `system` はA1セル、`user` はB1セル、`assistant` はC1セルに入力してください。

|system|user|assistant|
|:-:|:-:|:-:|
|あなたはスタックチャンです。おはなしをしよう。|こんにちは|こんにちは僕の名前はスタックチャンです|
|あなたはスタックチャンです。おはなしをしよう。|{任意の質問}|{任意の質問に対応する解答}|
|︙|︙|︙|


### 1. ライブラリのインポート・関数定義

In [ ]:
%%capture
# BERT系
!pip install transformers
!pip install sentencepiece
# MeCab
!pip install mecab-python3
# unidicを使う
!pip install unidic
!python -m unidic download
# ipadicを使う
!pip install ipadic
# データフレーム
!pip install openpyxl

In [ ]:
#
# 当セルはランタイムにつき1回の実行でOKです
#

# ライブラリのインポート
import numpy as np
import pandas as pd
import json
import torch
import sentencepiece
import MeCab
import unidic
import ipadic
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForMaskedLM

# BERTモデルのインスタンス化
pipe = pipeline("fill-mask", model="nlp-waseda/roberta-base-japanese")
tokenizer = AutoTokenizer.from_pretrained("nlp-waseda/roberta-base-japanese")
model = AutoModelForMaskedLM.from_pretrained("nlp-waseda/roberta-base-japanese")


# 分かち書きを行う関数
def wakachi(text = '早稲田大学で自然言語処理を専攻する。'): # 引数に入っている初期値は想定する入力
  # 分かち書き
  DATA = np.array([['','']])
  mecab = MeCab.Tagger(ipadic.MECAB_ARGS)
  tokens = mecab.parse(text)

  for token in tokens.split('\n'):
    data = token.split('\t')
    try:
      # print([[data[0],data[1].split(',')[0]]])
      DATA = np.append(DATA, [[data[0],data[1].split(',')[0]]],0)
    except:
      pass
  DATA=DATA[1:]

  texts = []
  for i in range(len(DATA.T[0])):
    if DATA.T[1][i] in ['名詞']:
      texts.append(' '.join(list(DATA.T[0][:i])+['[MASK]']+list(DATA.T[0][i+1:])))
  return texts

# BERTによる予測を行う関数
def word_prediction(text = "早稲田 大学 で 自然 言語 処理 を [MASK] する 。",add_num = 10): # 引数に入っている初期値は想定する入力
  input_ids = tokenizer.encode(text, return_tensors="pt")
  mask_idx = input_ids.tolist()[0].index(tokenizer.mask_token_id)
  with torch.no_grad():
      prediction = model(input_ids).logits

  # Get the probabilities using softmax
  probs = torch.nn.functional.softmax(prediction[0, mask_idx], dim=0)

  # Get the top 10 predicted token ids and their probabilities
  top_10_token_ids = torch.topk(probs, add_num).indices
  top_10_probs = torch.topk(probs, add_num).values

  predicted_words = [tokenizer.decode([token_id]) for token_id in top_10_token_ids]

  # Zip the words with their probabilities and print them
  for word, prob in zip(predicted_words, top_10_probs):
      pass
      #print(f"{word}: {prob.item():.5f}")

  return predicted_words  # zip(predicted_words, top_10_probs)

#### 2.実行

In [ ]:
file_path = '..' # 同一でない場合は適宜変更する。
DATA = pd.read_excel(f'{file_path}/FineTuningData.xlsx') # ファイル名は適宜変更する。

for i in range(len(DATA['user'])):
  #input(DATA['user'][i])

  # 言い換え文を作成
  new_keywords = []
  for text in wakachi(text = DATA['user'][i]): # userの入力文を与える
    #input(text)
    for word in word_prediction(text,30):
      if word not in  ['。','、','?','!','.','...','......','−','–','――','「','」','～','(',')','『','”',':']:
        new_keywords.append( text.replace('[MASK]', word).replace(' ', '') )


  #new_keywords = ['a','b','c']

  # 拡張したデータを追加
  for new_keyword in new_keywords:
    new_data = pd.DataFrame([{
        'system':DATA['system'][i],
        'user':new_keyword, # 書き換えた入力を加える
        'assistant':DATA['assistant'][i]
        }])
    DATA = pd.concat([DATA, new_data], ignore_index=True)


# チューニングデータを作成
TunigData = []
for i in DATA.T:
  append_json = {
      'messages': [
          {'role': 'system','content': DATA.T[i]['system']},
           {'role': 'user', 'content': DATA.T[i]['user']},
            {'role': 'assistant','content': DATA.T[i]['assistant']}
      ]}
  TunigData.append(append_json)

with open('TuningData.jsonl', 'w') as file:
  for entry in TunigData:
      file.write(json.dumps(entry) + '\n')

DATA